<a href="https://colab.research.google.com/github/pscabral/IFMASOFTEX_1/blob/main/ModeloYOLO8_2.00.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mega.py

from mega import Mega

# Crie uma instância da classe Mega
mega = Mega()

# Cole o link completo do Mega.nz (incluindo a chave)
url = "https://mega.nz/file/tu0GXQzL#BGoVzch2_6s2EEQYLfkwpct1Uq-ZrRL2sHCOY54vohs"

# Use o link completo para fazer o download
mega.download_url(url)

!unzip vegetables.v4i.yolov8.zip

In [ ]:
!pip install ultralytics

In [3]:
from PIL import Image
import os
import numpy as np
import torch
import random
import pathlib
from torch.utils.data import Dataset
import json
from torchvision import transforms, utils
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO, utils

In [4]:
model = YOLO("yolov8n.pt")

100%|██████████| 6.23M/6.23M [00:00<00:00, 132MB/s]


In [5]:
class Config:
    CHECKPOINT_DIR = 'checkpoints'
    EXPERIMENT_NAME = '/content/vegetables'
    DATA_DIR = '../content/vegetables'
    TRAIN_IMAGES_DIR = 'train/images'
    TRAIN_LABELS_DIR = 'train/labels'
    VAL_IMAGES_DIR = 'valid/images'
    VAL_LABELS_DIR = 'valid/labels'
    TEST_IMAGES_DIR = 'test/images'
    TEST_LABELS_DIR = 'test/labels'
    CLASSES = ['apple', 'banana', 'bell_pepper', 'carrot', 'cauliflower', 'chillies', 'cucumber', 'garlic', 'grapes', 'mango', 'mushroom', 'onion', 'orange', 'pear', 'potato', 'promegranate', 'tomato', 'watermelon']
    NUM_CLASSES = len(CLASSES)
    DATALOADER_PARAMS = {'batch_size': 16, 'num_workers': 2}

In [6]:
config = Config()

In [ ]:
train_images_dir = os.path.join(config.DATA_DIR, config.TRAIN_IMAGES_DIR)
train_labels_dir = os.path.join(config.DATA_DIR, config.TRAIN_LABELS_DIR)

num_images = 5

image_files = [f for f in pathlib.Path(train_images_dir).iterdir() if f.is_file()]

random.shuffle(image_files)

selected_image_files = image_files[:num_images]

colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255), (255, 0, 255),
          (128, 0, 0), (0, 128, 0), (0, 0, 128), (128, 128, 0), (0, 128, 128), (128, 0, 128),
          (64, 0, 0), (0, 64, 0), (0, 0, 64), (64, 64, 0), (0, 64, 64), (64, 0, 64)]

for selected_image_file in selected_image_files:
    demo_image = selected_image_file

    demo_label = pathlib.Path(train_labels_dir) / f"{selected_image_file.stem}.txt"

    image = cv2.imread(str(demo_image))

    class_list = config.CLASSES

    height, width, _ = image.shape

    with open(demo_label, "r") as file1:
        for line in file1.readlines():

            split = line.split(" ")

            class_id = int(split[0])

            if 0 <= class_id < len(colors):

                color = colors[class_id]
                clazz = class_list[class_id]

                x, y, w, h = map(float, split[1:5])

                box = [int(x * width), int(y * height), int(w * width), int(h * height)]

                cv2.rectangle(image, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), color, 2)

                cv2.rectangle(image, (box[0], box[1] - 20), (box[0] + box[2], box[1]), color, -1)

                cv2.putText(image, clazz, (box[0], box[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

            else:
                print(f"ID da classe inválido: {class_id}. Certifique-se de que está dentro da faixa [0, {len(colors) - 1}].")

    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    image = cv2.resize(image, (600, 600))
    plt.show()

In [41]:
!cat /content/vegetables/data.yaml

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 18
names: ['apple', 'banana', 'bell_pepper', 'carrot', 'cauliflower', 'chillies', 'cucumber', 'garlic', 'grapes', 'mango', 'mushroom', 'onion', 'orange', 'pear', 'potato', 'promegranate', 'tomato', 'watermelon']

roboflow:
  workspace: news
  project: vegetables-mr0jj
  version: 4
  license: CC BY 4.0
  url: https://universe.roboflow.com/news/vegetables-mr0jj/dataset/4

In [40]:
! yolo task=detect mode=train model=yolov8n.pt imgsz=640 data=/content/vegetables/data.yaml epochs=20 batch=8 name=yolov8n_custom

Ultralytics YOLOv8.0.217 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/vegetables/data.yaml, epochs=20, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_custom2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=Fa

In [42]:
from ultralytics import YOLO

# Carregar um modelo
model = YOLO('yolov8n.pt')  # carregar um modelo oficial
model = YOLO('/content/runs/detect/yolov8n_custom2/weights/best.pt')  # carregar um modelo personalizado

# Validar o modelo
metrics = model.val()  # nenhum argumento necessário, conjunto de dados e configurações lembrados
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # uma lista contém map50-95 de cada categoria


Ultralytics YOLOv8.0.217 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3009158 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/vegetables/valid/labels.cache... 256 images, 0 backgrounds, 0 corrupt: 100%|██████████| 256/256 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:23<00:00,  1.47s/it]


                   all        256        636      0.729      0.669      0.725      0.403
                 apple        256         76      0.689      0.882      0.856      0.457
                banana        256         25      0.823       0.96      0.965      0.544
           bell_pepper        256         74      0.864      0.942      0.962      0.589
                carrot        256         27      0.505      0.556      0.625      0.359
           cauliflower        256         15          1      0.987      0.995      0.577
              chillies        256         16      0.402      0.126      0.325      0.125
              cucumber        256         23       0.61       0.87       0.85      0.393
                garlic        256          3          1          0      0.677      0.507
                grapes        256         49       0.91      0.959      0.971      0.492
                 mango        256         17      0.235      0.294      0.212      0.143
              mushroo

array([    0.45741,     0.54444,     0.58877,      0.3587,     0.57702,     0.12492,     0.39323,     0.50673,     0.49167,     0.14345,     0.12823,     0.55107,     0.50237,     0.42987,     0.47486,           0,     0.39375,     0.58057])

In [ ]:
!yolo detect predict model='/content/runs/detect/yolov8n_custom2/weights/best.pt' source='/content/tresfrutas03.jpg'

In [ ]:
!yolo export model='/content/runs/detect/yolov8n_custom2/weights/best.pt' format=onnx

In [73]:
from ultralytics import YOLO
import cv2

model = YOLO("../content/runs/detect/yolov8n_custom2/weights/best.onnx")

im2 = cv2.imread("/content/tresfrutas01.jpg")

# Resize the image to the expected dimensions (640x640)
im2 = cv2.resize(im2, (640, 640))

results = model.predict(source=im2, save=True, save_txt=True, imgsz=640)  # Update imgsz to the resized dimensions

for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    probs = result.probs  # Class probabilities for classification outputs


Loading ../content/runs/detect/yolov8n_custom2/weights/best.onnx for ONNX Runtime inference...

0: 640x640 1 apple, 1 banana, 1 pear, 92.2ms
Speed: 2.5ms preprocess, 92.2ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict12
1 label saved to runs/detect/predict12/labels


In [74]:
import cv2
import sys
import os
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # path to model file
cap = cv2.VideoCapture(0)  # path to video file or webcam
crop_filename = "crop"  # set the crop file name
crop_count = 0

if not cap.isOpened():
    print("Error reading video file")
    sys.exit()

if not os.path.exists("crop"):
    os.mkdir("crop")

while cap.isOpened():

    success, frame = cap.read()
    if success:
        results = model.predict(frame, verbose=False)
        boxes = results[0].boxes.xyxy.cpu()
        for box in boxes:
            crop_count += 1
            crop_object = frame[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
            cv2.imwrite(os.path.join("crop", crop_filename+ f"_{crop_count}.png"), crop_object)

        cv2.imshow("YOLOv8 Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

Error reading video file


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
